# AIE425 Intelligent Recommender Systems, Fall Semester 25/26
Assignment 1: Neighborhood CF & Clustering in CF
| Name | ID | Part |
| ------ | -----| -----|
| Youssef Husseiny | 222101943 | Section 3 |
| Habiba Ahmed | 222100471 | Section 2 |
| Seif Amr Abdelhafez | 222102312 | Section 1 |

## Part 2: Item-Based Collaborative Filtering Requirements and Questions:


In [3]:
import pandas as pd
import numpy as np

In [4]:
m = pd.read_csv('../../dataset/movies.csv')
r = pd.read_csv('../../dataset/ratings.csv')

In [5]:
import os

In [6]:
os.getcwd()

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\section2_neighborhood_cf\\part2_item_based_cf'

In [7]:
import os
import sys

utils_path = os.path.abspath(os.path.join(os.getcwd(), '../..', 'utils'))

In [8]:
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [9]:
print(f"Calculated utils_path: {utils_path}")
print(f"Is utils_path in sys.path: {utils_path in sys.path}")

Calculated utils_path: d:\My_Laptop\University\GU\AI Science\Year 4\Semester 1\Intelligent Recommender System\Assignments\Assignment1\utils
Is utils_path in sys.path: True


In [10]:
os.listdir(utils_path)

['AdjustedCosine_ItemSim.py',
 'Cosine_Sim.py',
 'MeanCenteredSimilarityRS.py',
 'Pearson_ItemSim.py',
 'Pearson_Sim.py',
 '__init__.py',
 '__pycache__']

In [11]:
utils_path

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\utils'

In [12]:
print(f"Original Data: {r['userId'].nunique()} Users, {r['movieId'].nunique()} Movies")

Original Data: 200948 Users, 84432 Movies


In [13]:
def read_ids_from_file(filepath):
    """Reads IDs from a file."""
    if not os.path.exists(filepath):
        print(f"Warning: File not found {filepath}")
        return []
    with open(filepath, 'r') as f:
        content = f.read().replace('\n', ',') 
        return [int(float(x)) for x in content.split(',') if x.strip().replace('.','',1).isdigit()]

In [14]:
target_users_ids = read_ids_from_file('../../Results/target_users.txt')
target_items_ids = read_ids_from_file('../../Results/target_items.txt')

In [15]:
print(f"Loaded Target Users: {target_users_ids}")
print(f"Loaded Target Items: {target_items_ids}")

Loaded Target Users: [79060, 183483, 55243]
Loaded Target Items: [203146, 230591]


In [16]:
top_movies = r['movieId'].value_counts().head(1200).index.tolist()
target_user_movies = r[r['userId'].isin(target_users_ids)]['movieId'].unique().tolist()
movies_to_keep = list(set(top_movies) | set(target_items_ids) | set(target_user_movies))
r_filtered = r[r['movieId'].isin(movies_to_keep)]

In [17]:
top_users = r_filtered['userId'].value_counts().head(110000).index.tolist()
users_who_rated_targets = r[r['movieId'].isin(target_items_ids)]['userId'].unique().tolist()
users_to_keep = list(set(top_users) | set(target_users_ids) | set(users_who_rated_targets))
r_filtered = r_filtered[r_filtered['userId'].isin(users_to_keep)]

In [18]:
item_matrix_df = r_filtered.pivot_table(index='movieId', columns='userId', values='rating')
item_matrix_np = item_matrix_df.values

item_ids = item_matrix_df.index.tolist()    
user_ids = item_matrix_df.columns.tolist()

In [19]:
print(f"Users: {r_filtered['userId'].nunique()} (criteria: >= 100,000)")
print(f"Products: {r_filtered['movieId'].nunique()} (criteria: >= 1,000)")
print(f"Ratings: {len(r_filtered)} (criteria: >= 1,000,000)")

Users: 110004 (criteria: >= 100,000)
Products: 1217 (criteria: >= 1,000)
Ratings: 17728480 (criteria: >= 1,000,000)


In [20]:
item_matrix_np.shape

(1217, 110004)

### Case study 1

#### 1. Apply item-based collaborative filtering using Cosine similarity with mean-centering.


In [21]:
from AdjustedCosine_ItemSim import AdjustedCosine_ItemBased_RS

In [47]:
adj_item_rs = AdjustedCosine_ItemBased_RS(item_matrix_np)
adj_item_rs.fit()

Item-Based Model fitted. Matrix Shape: (1217, 110004)


In [48]:
def get_adjusted_cosine_neighbors(item_id, top_n=10):
    """Get Top-N similar items using Adjusted Cosine Similarity."""
    
    # --- FIX: Check if item exists in matrix first ---
    if item_id not in item_ids:
        print(f"Target Movie {item_id} is missing from the matrix (no ratings found).")
        return

    i_idx = item_ids.index(item_id)
    
    sims = adj_item_rs.compute_similarities(i_idx)
    
    sorted_idx = np.argsort(sims)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<5} | {'Similar Movie ID':<18} | {'Adj Cosine':<12}")
    print("-" * 50)
    
    count = 0
    for idx in sorted_idx:
        score = sims[idx]
        if score <= 0: continue
            
        similar_id = item_ids[idx]
        print(f"{count+1:<5} | {similar_id:<18} | {score:.6f}")
        
        count += 1
        if count >= 10: break
    
    if count == 0:
        print("  No similar movies found.")

In [49]:
get_adjusted_cosine_neighbors(target_items_ids[0])


Target Movie: 203146
Rank  | Similar Movie ID   | Adj Cosine  
--------------------------------------------------
1     | 195159             | 0.040053
2     | 1061               | 0.026904
3     | 5064               | 0.026715
4     | 8638               | 0.025524
5     | 671                | 0.025204
6     | 799                | 0.024908
7     | 40819              | 0.024345
8     | 2707               | 0.023868
9     | 60756              | 0.023237
10    | 348                | 0.022619


In [50]:
get_adjusted_cosine_neighbors(target_items_ids[1])  


Target Movie: 230591
Rank  | Similar Movie ID   | Adj Cosine  
--------------------------------------------------
1     | 208703             | 0.013851
2     | 2087               | 0.011523
3     | 2096               | 0.011220
4     | 207313             | 0.011205
5     | 2080               | 0.009831
6     | 969                | 0.009530
7     | 1947               | 0.009325
8     | 122914             | 0.008754
9     | 910                | 0.008554
10    | 1088               | 0.006847


#### 2. Identify the top 20% of similar items for each target item.


In [51]:
def get_item_top_20(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return []
    
    i_idx = item_ids.index(item_id)
    
    all_sims = adj_item_rs.compute_similarities(i_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    total_items = len(item_ids)
    top_20_count = int(total_items * 0.20)
    
    top_20_indices = sorted_indices[:top_20_count]
    
    top_20_items = []
    for idx in top_20_indices:
        score = all_sims[idx]
        similar_item_id = item_ids[idx]
        
        if score > 0:
            top_20_items.append((similar_item_id, score))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Total Movies in Matrix: {total_items}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Items with Positive Sim in Top 20%: {len(top_20_items)}")
    
    print(f"\n{'- Preview of Top 10 Similar Movies -':^50}")
    print(f"{'Rank':<5} | {'Movie ID':<15} | {'Adj Cosine':<10}")
    print("-" * 50)
    
    for rank, (sim_id, score) in enumerate(top_20_items[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.5f}")
        
    print("-" * 50)
    return top_20_items

In [52]:
top_20_items_1 = get_item_top_20(target_items_ids[0])


Target Movie: 203146
Total Movies in Matrix: 1217
Top 20% Threshold Count: 243
Items with Positive Sim in Top 20%: 243

       - Preview of Top 10 Similar Movies -       
Rank  | Movie ID        | Adj Cosine
--------------------------------------------------
1     | 195159          | 0.04005
2     | 1061            | 0.02690
3     | 5064            | 0.02671
4     | 8638            | 0.02552
5     | 671             | 0.02520
6     | 799             | 0.02491
7     | 40819           | 0.02434
8     | 2707            | 0.02387
9     | 60756           | 0.02324
10    | 348             | 0.02262
--------------------------------------------------


In [53]:
top_20_items_2 = get_item_top_20(target_items_ids[1])


Target Movie: 230591
Total Movies in Matrix: 1217
Top 20% Threshold Count: 243
Items with Positive Sim in Top 20%: 29

       - Preview of Top 10 Similar Movies -       
Rank  | Movie ID        | Adj Cosine
--------------------------------------------------
1     | 208703          | 0.01385
2     | 2087            | 0.01152
3     | 2096            | 0.01122
4     | 207313          | 0.01120
5     | 2080            | 0.00983
6     | 969             | 0.00953
7     | 1947            | 0.00932
8     | 122914          | 0.00875
9     | 910             | 0.00855
10    | 1088            | 0.00685
--------------------------------------------------


#### 3. Predict missing ratings using these items.

In [54]:
def generate_item_recommendations(uid,K = 10):
    if uid not in user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = user_ids.index(uid)
    user_ratings = item_matrix_np[:, u_idx]
    unrated_item_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_item_indices)}")
    print("Calculating Item-Based predictions... (this might take a minute)")
    
    recommendations = []
    
    for i_idx in unrated_item_indices:
        try:
            pred_val = adj_item_rs.pred(u_idx=u_idx, i_idx=i_idx, k=K)
            
            if pred_val > 0:
                item_id = item_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Item-Based):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Predicted Rating':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations

In [55]:
item_recs_u1 = generate_item_recommendations(target_users_ids[0])


Target User: 79060
Total Unrated Items: 1197
Calculating Item-Based predictions... (this might take a minute)

  Top 5 Recommended Movies (Item-Based):
  Rank  | Movie ID     | Predicted Rating
  ----------------------------------------
  1     | 89745        | 5.0000
  2     | 122926       | 4.9392
  3     | 2872         | 4.7118
  4     | 916          | 4.3211
  5     | 1663         | 4.3206
------------------------------------------------------------


In [56]:
item_recs_u2 = generate_item_recommendations(target_users_ids[1])


Target User: 183483
Total Unrated Items: 1195
Calculating Item-Based predictions... (this might take a minute)

  Top 5 Recommended Movies (Item-Based):
  Rank  | Movie ID     | Predicted Rating
  ----------------------------------------
  1     | 161          | 4.9915
  2     | 164909       | 4.9845
  3     | 92259        | 4.9782
  4     | 109487       | 4.9753
  5     | 112552       | 4.9712
------------------------------------------------------------


#### 4. Compute DF and DS.


In [57]:
from AdjustedCosine_ItemSim import Discounted_AdjustedCosine_ItemBased_RS


In [58]:
d_adj_item_rs = Discounted_AdjustedCosine_ItemBased_RS(item_matrix_np)
d_adj_item_rs.fit()

Item-Based Model fitted. Matrix Shape: (1217, 110004)


In [59]:
total_users = item_matrix_np.shape[1] 
beta_percentage = 0.30
beta_threshold = int(np.ceil(total_users * beta_percentage))

In [60]:
def get_discounted_similar_items(item_id):
    """Get Top-N similar items using Discounted Adjusted Cosine Similarity."""
        
    i_idx = item_ids.index(item_id)
    
    raw_sims, counts, dfs, dses = d_adj_item_rs.get_discounted_stats(i_idx, beta_threshold)
    
    sorted_indices = np.argsort(raw_sims)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<4} | {'Similar Item':<18} | {'Common Users':<12} | {'DF':<6} | {'Adj Cos':<10} | {'-->':<3} | {'DS (Disc)':<10}")
    print("-" * 90)
    
    count = 0
    for idx in sorted_indices:
        if raw_sims[idx] <= 0: continue
        
        sim_id = item_ids[idx]
        n_raw = raw_sims[idx]
        n_count = int(counts[idx])
        n_df = dfs[idx]
        n_ds = dses[idx]
        
        print(f"{count+1:<4} | {sim_id:<18} | {n_count:<12} | {n_df:.4f} | {n_raw:.6f} | --> | {n_ds:.6f}")
        
        count += 1
        if count >= 10: break
        
    if count == 0:
        print("  No similar items found.")

In [62]:
get_discounted_similar_items(target_items_ids[0])  


Target Movie: 203146
Rank | Similar Item       | Common Users | DF     | Adj Cos    | --> | DS (Disc) 
------------------------------------------------------------------------------------------
1    | 195159             | 3            | 0.0001 | 0.040053 | --> | 0.000004
2    | 1061               | 2            | 0.0001 | 0.026904 | --> | 0.000002
3    | 5064               | 2            | 0.0001 | 0.026715 | --> | 0.000002
4    | 8638               | 2            | 0.0001 | 0.025524 | --> | 0.000002
5    | 671                | 2            | 0.0001 | 0.025204 | --> | 0.000002
6    | 799                | 2            | 0.0001 | 0.024908 | --> | 0.000002
7    | 40819              | 2            | 0.0001 | 0.024345 | --> | 0.000001
8    | 2707               | 2            | 0.0001 | 0.023868 | --> | 0.000001
9    | 60756              | 2            | 0.0001 | 0.023237 | --> | 0.000001
10   | 348                | 2            | 0.0001 | 0.022619 | --> | 0.000001


In [63]:
get_discounted_similar_items(target_items_ids[1])  


Target Movie: 230591
Rank | Similar Item       | Common Users | DF     | Adj Cos    | --> | DS (Disc) 
------------------------------------------------------------------------------------------
1    | 208703             | 1            | 0.0000 | 0.013851 | --> | 0.000000
2    | 2087               | 1            | 0.0000 | 0.011523 | --> | 0.000000
3    | 2096               | 1            | 0.0000 | 0.011220 | --> | 0.000000
4    | 207313             | 1            | 0.0000 | 0.011205 | --> | 0.000000
5    | 2080               | 1            | 0.0000 | 0.009831 | --> | 0.000000
6    | 969                | 1            | 0.0000 | 0.009530 | --> | 0.000000
7    | 1947               | 1            | 0.0000 | 0.009325 | --> | 0.000000
8    | 122914             | 1            | 0.0000 | 0.008754 | --> | 0.000000
9    | 910                | 1            | 0.0000 | 0.008554 | --> | 0.000000
10   | 1088               | 1            | 0.0000 | 0.006847 | --> | 0.000000


#### 5. Select top 20% items using DS.


In [64]:
total_users = item_matrix_np.shape[1] 
beta_threshold = int(np.ceil(total_users * 0.30))

In [65]:
def get_item_top_20_discounted(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return []
    
    i_idx = item_ids.index(item_id)
    _, _, _, dses = d_adj_item_rs.get_discounted_stats(i_idx, beta_threshold)
    
    sorted_indices_ds = np.argsort(dses)[::-1]
    
    total_items = len(item_ids)
    top_20_count = int(total_items * 0.20)
    
    top_20_indices_ds = sorted_indices_ds[:top_20_count]
    
    top_20_items_ds = []
    for idx in top_20_indices_ds:
        sim_id = item_ids[idx]
        score_ds = dses[idx]
        
        if score_ds > 0:
            top_20_items_ds.append((sim_id, score_ds))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Items with Positive DS in Top 20%: {len(top_20_items_ds)}")
    
    print(f"\n{'- Preview of Top 10 (Discounted) -':^50}")
    print(f"{'Rank':<5} | {'Movie ID':<15} | {'Discounted Sim':<15}")
    print("-" * 50)
    
    for rank, (sim_id, score) in enumerate(top_20_items_ds[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.6f}")
        
    print("-" * 50)
    return top_20_items_ds

In [66]:
item_top_20_ds_1 = get_item_top_20_discounted(target_items_ids[0])


Target Movie: 203146
Top 20% Threshold Count: 243
Items with Positive DS in Top 20%: 243

        - Preview of Top 10 (Discounted) -        
Rank  | Movie ID        | Discounted Sim 
--------------------------------------------------
1     | 195159          | 0.000004
2     | 4700            | 0.000002
3     | 56782           | 0.000002
4     | 1061            | 0.000002
5     | 5064            | 0.000002
6     | 60684           | 0.000002
7     | 51255           | 0.000002
8     | 8638            | 0.000002
9     | 671             | 0.000002
10    | 799             | 0.000002
--------------------------------------------------


In [67]:
item_top_20_ds_2 = get_item_top_20_discounted(target_items_ids[1])


Target Movie: 230591
Top 20% Threshold Count: 243
Items with Positive DS in Top 20%: 29

        - Preview of Top 10 (Discounted) -        
Rank  | Movie ID        | Discounted Sim 
--------------------------------------------------
1     | 208703          | 0.000000
2     | 2087            | 0.000000
3     | 2096            | 0.000000
4     | 207313          | 0.000000
5     | 2080            | 0.000000
6     | 969             | 0.000000
7     | 1947            | 0.000000
8     | 122914          | 0.000000
9     | 910             | 0.000000
10    | 1088            | 0.000000
--------------------------------------------------


#### 6. Use these for updated rating predictions.


In [70]:
from tqdm import tqdm
def generate_item_discounted_recommendations(uid,K = 10):
    if uid not in user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = user_ids.index(uid)
    
    user_ratings = d_adj_item_rs.matrix[:, u_idx]
    unrated_item_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_item_indices)}")
    print("Calculating Discounted Item-Based predictions...")
    
    recommendations = []
    
    for i_idx in tqdm(unrated_item_indices):
        try:
            pred_val = d_adj_item_rs.pred_discounted(u_idx, i_idx, K, beta_threshold)
            if pred_val > 0:
                item_id = item_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Discounted Item-Based):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Disc Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations

In [71]:
item_disc_recs_u1 = generate_item_discounted_recommendations(target_users_ids[0])


Target User: 79060
Total Unrated Items: 1197
Calculating Discounted Item-Based predictions...


  0%|          | 0/1197 [00:00<?, ?it/s]

  7%|▋         | 85/1197 [14:44<3:12:46, 10.40s/it]


KeyboardInterrupt: 

In [ ]:
item_disc_recs_u2 = generate_item_discounted_recommendations(target_users_ids[1])

#### 7. Compare similarity lists from steps 2 and 5. Provide commentary.

In [ ]:
top_20_items_1

In [ ]:
top_20_items_1_df = pd.DataFrame(top_20_items_1, columns=['movieId', 'discounted_similarity'])
top_20_items_1_df

In [ ]:
item_top_20_ds_1_df = pd.DataFrame(item_top_20_ds_1, columns=['movieId', 'discounted_similarity'])
item_top_20_ds_1_df

In [ ]:
top_20_items_1_compare = pd.merge(top_20_items_1_df, item_top_20_ds_1_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_1_compare

In [ ]:
top_20_items_2_df = pd.DataFrame(top_20_items_2, columns=['movieId', 'discounted_similarity'])
top_20_items_2_df

In [ ]:
item_top_20_ds_2_df = pd.DataFrame(item_top_20_ds_2, columns=['movieId', 'discounted_similarity'])
item_top_20_ds_2_df

In [ ]:
top_20_items_2_compare = pd.merge(top_20_items_2_df, item_top_20_ds_2_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_2_compare

##### Comment


The Discounted list is far more robust. It prioritizes items where the similarity is statistically significant, ensuring that the recommendation is based on a consensus of many users rather than the coincidence of a few.

#### 8. Compare predicted ratings from steps 3 and 6. Discuss.

In [ ]:
item_recs_u1_df = pd.DataFrame(item_recs_u1, columns=['movieId', 'predicted_rating'])
item_recs_u1_df

In [ ]:
item_disc_recs_u1_df = pd.DataFrame(item_disc_recs_u1, columns=['movieId', 'predicted_rating'])
item_disc_recs_u1_df

In [ ]:
item_recs_u2_df = pd.DataFrame(item_recs_u2, columns=['movieId', 'predicted_rating'])
item_recs_u2_df

In [ ]:
item_disc_recs_u2_df = pd.DataFrame(item_disc_recs_u2, columns=['movieId', 'predicted_rating'])
item_disc_recs_u2_df

##### Discussion

The predicted ratings generated using the Discounted Similarity (Task 6) are generally lower or more conservative compared to the predictions from the Raw Similarity (Task 3). because of Filtering Outliers and discount factor weighting

#### 9. Give your comments in a separate section in your report.

- The few common users Problem: Adjusted Cosine Similarity calculates the angle between two vectors. If the vectors are very short (e.g., only 2 common users), it is statistically easy for them to align perfectly by chance.Example: If User A rates both Movie X and Movie Y as 5.0, and User B rates both as 1.0, the math sees a perfect pattern match (Sim = 1.0), even though we only have 2 data points.
- Lack of Variance: Without enough common users, the algorithm cannot distinguish between true preference correlation and random coincidence.
- Role of Discounting: This confirms why computing DF and DS (Task 4) is critical. By setting a threshold (e.g., $\beta = 30\%$), we force the algorithm to treat low-support correlations as "weak," regardless of how high their raw cosine score is. This prevents the system from recommending obscure items based on insufficient evidence.

### Case study 2

In [33]:
from Pearson_ItemSim import Pearson_ItemBased_RS, Discounted_Pearson_ItemBased_RS

In [34]:
pcc_item_rs = Pearson_ItemBased_RS(item_matrix_np)
pcc_item_rs.fit()

Pearson Item-Based Model fitted. Means computed for 1217 items.


#### 1- Use PCC to compute similarity between target items.

In [35]:
def get_item_pcc_neighbors(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return
    
    i_idx = item_ids.index(item_id)
    
    # Compute Pearson Similarity
    sims = pcc_item_rs.compute_similarities(i_idx)
    
    sorted_idx = np.argsort(sims)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<5} | {'Similar Movie ID':<18} | {'PCC Sim':<12}")
    print("-" * 50)
    
    count = 0
    for idx in sorted_idx:
        score = sims[idx]
        if score <= 0: continue
            
        similar_id = item_ids[idx]
        print(f"{count+1:<5} | {similar_id:<18} | {score:.6f}")
        
        count += 1
        if count >= 10: break
    
    if count == 0:
        print("  No similar movies found.")

In [36]:
for item_id in target_items_ids:
    if item_id in item_ids:
        idx = item_ids.index(item_id)
        # Count non-NaN ratings for this item
        count = np.sum(~np.isnan(item_matrix_np[idx]))
        print(f"Movie {item_id}: Rated by {count} users in the final matrix.")
    else:
        print(f"Movie {item_id}: Not in matrix.")

Movie 203146: Rated by 3 users in the final matrix.
Movie 230591: Rated by 1 users in the final matrix.


In [37]:
get_item_pcc_neighbors(target_items_ids[0])



Target Movie: 203146
Rank  | Similar Movie ID   | PCC Sim     
--------------------------------------------------
  No similar movies found.


In [38]:
get_item_pcc_neighbors(target_items_ids[1])


Target Movie: 230591
Rank  | Similar Movie ID   | PCC Sim     
--------------------------------------------------
  No similar movies found.


#### 2- Identify the top 20% most similar items.

In [39]:
def get_item_pcc_top_20(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return []
    
    i_idx = item_ids.index(item_id)
    all_sims = pcc_item_rs.compute_similarities(i_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    total_items = len(item_ids)
    top_20_count = int(total_items * 0.20)
    
    top_20_indices = sorted_indices[:top_20_count]
    
    top_20_items = []
    for idx in top_20_indices:
        score = all_sims[idx]
        if score > 0:
            top_20_items.append((item_ids[idx], score))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Items with Positive PCC in Top 20%: {len(top_20_items)}")
    
    print(f"\n{'- Preview of Top 10 -':^40}")
    print(f"{'Rank':<5} | {'Movie ID':<15} | {'PCC Sim':<10}")
    print("-" * 40)
    
    for rank, (sim_id, score) in enumerate(top_20_items[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.5f}")
        
    return top_20_items

In [40]:
pcc_top_20_1 = get_item_pcc_top_20(target_items_ids[0])


Target Movie: 203146
Items with Positive PCC in Top 20%: 0

         - Preview of Top 10 -          
Rank  | Movie ID        | PCC Sim   
----------------------------------------


In [41]:
pcc_top_20_2 = get_item_pcc_top_20(target_items_ids[1])


Target Movie: 230591
Items with Positive PCC in Top 20%: 0

         - Preview of Top 10 -          
Rank  | Movie ID        | PCC Sim   
----------------------------------------


#### 3- Predict the missing ratings.


In [42]:
from tqdm import tqdm
def generate_pcc_item_recs(uid, K=10):
    if uid not in user_ids: return []
    
    u_idx = user_ids.index(uid)
    
    user_ratings = item_matrix_np[:, u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_indices)}")
    print("Calculating PCC predictions...")
    
    recs = []
    for i_idx in tqdm(unrated_indices):
        try:
            val = pcc_item_rs.pred(u_idx, i_idx, K)
            if val > 0:
                recs.append((item_ids[i_idx], val))
        except: continue
            
    recs.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n  Top 5 Recommended Movies (PCC Item-Based):")
    print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Prediction':<15}")
    print("  " + "-" * 40)
    
    for rank, (movie_id, val) in enumerate(recs[:5]):
        print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
        
    return recs

In [ ]:
pcc_recs_u1 = generate_pcc_item_recs(target_users_ids[0])



Target User: 79060
Total Unrated Items: 1197
Calculating PCC predictions...


  1%|          | 11/1197 [01:11<1:58:30,  6.00s/it]

In [ ]:
pcc_recs_u2 = generate_pcc_item_recs(target_users_ids[1])


In [ ]:
pcc_recs_u3 = generate_pcc_item_recs(target_users_ids[2])

#### 4- Compute DF and DS using threshold B.


In [ ]:
d_pcc_item_rs = Discounted_Pearson_ItemBased_RS(item_matrix_np)
d_pcc_item_rs.fit()

total_users = item_matrix_np.shape[1]
beta_threshold = int(np.ceil(total_users * 0.30))
print(f"Beta Threshold (Users): {beta_threshold}")

In [ ]:
def get_item_pcc_discounted_stats(item_id):
    if item_id not in item_ids: return
    i_idx = item_ids.index(item_id)
    
    raw, counts, dfs, dses = d_pcc_item_rs.get_discounted_stats(i_idx, beta_threshold)
    sorted_indices = np.argsort(raw)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<4} | {'Similar Item':<15} | {'Common':<8} | {'DF':<6} | {'PCC':<8} | {'DS':<8}")
    print("-" * 80)
    
    count = 0
    for idx in sorted_indices:
        if raw[idx] <= 0: continue
        print(f"{count+1:<4} | {item_ids[idx]:<15} | {int(counts[idx]):<8} | {dfs[idx]:.4f} | {raw[idx]:.4f} | {dses[idx]:.4f}")
        count += 1
        if count >= 10: break


In [ ]:
get_item_pcc_discounted_stats(target_items_ids[0])

In [ ]:
get_item_pcc_discounted_stats(target_items_ids[1])

#### 5- Select the top 20% items based on discounted similarity.

In [ ]:
def get_item_pcc_disc_top20(item_id):
    if item_id not in item_ids: return []
    i_idx = item_ids.index(item_id)
    
    _, _, _, dses = d_pcc_item_rs.get_discounted_stats(i_idx, beta_threshold)
    
    sorted_idx = np.argsort(dses)[::-1]
    top_20_indices = sorted_idx[:int(len(item_ids)*0.2)]
    
    res = []
    for idx in top_20_indices:
        if dses[idx] > 0:
            res.append((item_ids[idx], dses[idx]))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Top 20% Threshold Count: {len(top_20_indices)}")
    print(f"Items with Positive DS: {len(res)}")
    
    print(f"\n{'- Preview of Top 10 (Discounted) -':^40}")
    for rank, (sim_id, score) in enumerate(res[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.5f}")
        
    return res

In [ ]:
pcc_disc_top_20_1 = get_item_pcc_disc_top20(target_items_ids[0])


In [ ]:
pcc_disc_top_20_2 = get_item_pcc_disc_top20(target_items_ids[1])

#### 6- Predict ratings again with this new selection.


In [ ]:
def generate_pcc_disc_item_recs(uid, K=10):
    if uid not in user_ids: return []
    u_idx = user_ids.index(uid)
    
    user_ratings = item_matrix_np[:, u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print("Calculating Discounted predictions...")
    
    recs = []
    for i_idx in unrated_indices:
        try:
            val = d_pcc_item_rs.pred_discounted(u_idx, i_idx, K, beta_threshold)
            if val > 0:
                recs.append((item_ids[i_idx], val))
        except: continue
            
    recs.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n  Top 5 Recommended Movies (Discounted PCC):")
    print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Disc Prediction':<15}")
    print("  " + "-" * 40)
    for rank, (movie_id, val) in enumerate(recs[:5]):
        print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
        
    return recs

In [ ]:
pcc_disc_recs_u1 = generate_pcc_disc_item_recs(target_users_ids[0])


In [ ]:
pcc_disc_recs_u2 = generate_pcc_disc_item_recs(target_users_ids[1])


In [ ]:
pcc_disc_recs_u3 = generate_pcc_disc_item_recs(target_users_ids[2])

#### 7 - Compare item lists from steps 2 and 5. Provide analysis.


In [ ]:
pcc_top_20_1

In [ ]:
pcc_top_20_1_df = pd.DataFrame(pcc_top_20_1, columns=['movieId', 'discounted_similarity'])
pcc_top_20_1_df

In [ ]:
pcc_disc_top_20_1_df = pd.DataFrame(pcc_disc_top_20_1, columns=['movieId', 'discounted_similarity'])
pcc_disc_top_20_1_df = pd.DataFrame(pcc_disc_top_20_1, columns=['movieId', 'discounted_similarity'])

In [ ]:
top_20_items_1_compare = pd.merge(pcc_top_20_1_df, pcc_disc_top_20_1_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_1_compare

In [ ]:
pcc_top_20_2_df = pd.DataFrame(pcc_top_20_2, columns=['movieId', 'discounted_similarity'])
pcc_top_20_2_df

In [ ]:
pcc_disc_top_20_2_df = pd.DataFrame(pcc_disc_top_20_2, columns=['movieId', 'discounted_similarity'])
pcc_disc_top_20_2_df

In [ ]:
top_20_items_2_compare = pd.merge(pcc_top_20_2_df, pcc_disc_top_20_2_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_2_compare

##### Analysis

Result: Both the "Raw Pearson" list (Step 2) and the "Discounted Pearson" list (Step 5) were empty for the target items, This confirms the "Variance Constraint" of Pearson Correlation. PCC requires at least 2 common users to calculate standard deviation. Since the target items are niche (rated by only 1-3 users total in the filtered matrix), they did not share $\ge 2$ users with any other item.

#### 8- Compare predictions from steps 3 and 6. Share insights.


In [ ]:
pcc_recs_u1_df = pd.DataFrame(pcc_recs_u1, columns=['movieId', 'predicted_rating'])
pcc_recs_u1_df

In [ ]:
pcc_disc_recs_u1_df = pd.DataFrame(pcc_disc_recs_u1, columns=['movieId', 'predicted_rating'])
pcc_disc_recs_u1_df

In [ ]:
pcc_recs_u1_compare = pd.merge(pcc_recs_u1_df, pcc_disc_recs_u1_df, on='movieId', suffixes=('_pcc', '_discounted'))
pcc_recs_u1_compare

In [ ]:
pcc_recs_u2_df = pd.DataFrame(pcc_recs_u2, columns=['movieId', 'predicted_rating']) 
pcc_recs_u2_df

In [ ]:
pcc_disc_recs_u2_df = pd.DataFrame(pcc_disc_recs_u2, columns=['movieId', 'predicted_rating'])   
pcc_disc_recs_u2_df

In [ ]:
pcc_recs_u2_compare = pd.merge(pcc_recs_u2_df, pcc_disc_recs_u2_df, on='movieId', suffixes=('_pcc', '_discounted'))
pcc_recs_u2_compare

In [ ]:
pcc_recs_u3_df = pd.DataFrame(pcc_recs_u3, columns=['movieId', 'predicted_rating']) 
pcc_recs_u3_df

In [ ]:
pcc_disc_recs_u3_df = pd.DataFrame(pcc_disc_recs_u3, columns=['movieId', 'predicted_rating'])   
pcc_disc_recs_u3_df

In [ ]:
pcc_recs_u3_compare = pd.merge(pcc_recs_u3_df, pcc_disc_recs_u3_df, on='movieId', suffixes=('_pcc', '_discounted'))
pcc_recs_u3_compare

##### Observation: 
Both the Raw Pearson (Step 3) and Discounted Pearson (Step 6) failed to generate predictions for the target items (returning empty lists or default values). This shows the "Cold Start" / "Sparsity" limitation of Item-Based Pearson Correlation. Because the target items were "niche" (rated by only 1-3 users), they did not satisfy the statistical requirements (Minimum Overlap $\ge 2$) needed for Pearson. The model remained silent rather than making a guess based on insufficient data.

#### 9- Give your comments in a separate section in your report


- Strictness of Pearson: Case Study 2 highlighted that Pearson is a "strict" metric. It protects the system from making wild guesses based on single-user overlaps (which Adjusted Cosine allowed in Case 1).
- The Trade-off: While stricter is usually better for accuracy, it is fatal for Coverage. For new or unpopular items (like our targets), Pearson completely fails to provide recommendations, whereas Adjusted Cosine can at least provide a "best guess" based on single shared users.
- Conclusion: For datasets with a "Long Tail" of unpopular items, Adjusted Cosine is often preferred over Pearson because it suffers less from this specific type of sparsity failure.

### Final Task for Part 2:
Compare the outcomes across Case Studies 1, 2, and 3, highlighting differences in prediction performance due to similarity measures and mean-centering.

1- Similarity Metric Robustness:
- Adjusted Cosine (Case 1): 
Proved to be Robust. It successfully identified neighbors for the sparse target items by leveraging even small overlaps (1 user). It handles User Bias by explicitly subtracting the User Mean from the item vector.
- Pearson Correlation (Case 2): Proved to be Fragile on sparse items. It failed to produce results because of the strict mathematical requirement for variance (Need $\ge 2$ common users).

2- Prediction Availability:

- Case 1: Generated predictions (though likely based on weak evidence).

- Case 2: Failed to generate predictions (Zero Coverage for these targets).

3- Final Recommendation:

- In a real-world system dealing with sparse data (like MovieLens), Adjusted Cosine Similarity (Item-Based) is generally the superior choice. It captures the same "User Bias" correction as Pearson but without the mathematical fragility that causes coverage issues for niche items. To fix the "weak evidence" problem of Adjusted Cosine, we apply Discounting (as done in Task 4), giving us the best of both worlds: Coverage + Reliability.